# Práctica 5 · Generación de Texto con GPT (Español)

**Autores:** Javier Ricardo Muñoz Castiollo · Yazmin Johana Garcia
**Trabajo:** Generador de leyendas ficticias con GPT-2 en español

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Ohtar10/icesi-nlp/blob/main/Sesion5/1-text-generation.ipynb)

En este notebook usaremos un modelo tipo GPT‑2 preentrenado en español para **generar texto** a partir de un *prompt* inicial. Luego haremos **ajuste fino (fine‑tuning)** con un pequeño corpus para observar cómo cambia el estilo de la generación.


## Resumen de la actividad

- Se configuró un entorno de generación con `DeepESP/gpt2-spanish`, asegurando compatibilidad con GPU CUDA y MPS.
- Se preparó un corpus propio de leyendas ficticias en formato JSONL y se realizó un EDA breve.
- Se tokenizó y dividió el conjunto en entrenamiento/validación para fine-tuning causal.
- Se afinó el modelo con `Trainer` y se generaron ejemplos usando prompts temáticos de leyendas.



#### Referencias
- Dataset de chistes: https://huggingface.co/datasets/mrm8488/CHISTES_spanish_jokes
- Radford et al. (2018): [Improving Language Understanding by Generative Pre‑Training](https://cdn.openai.com/research-covers/language-unsupervised/language_understanding_paper.pdf)
- Libro: *Natural Language Processing with Transformers* (O'Reilly) — Cap. 5
- Modelo: [GPT‑2 Spanish (DeepESP)](https://huggingface.co/DeepESP/gpt2-spanish)
- Guía: [Fine‑tune a non‑English GPT‑2 model with Hugging Face](https://www.philschmid.de/fine-tune-a-non-english-gpt-2-model-with-huggingface)



## GPT  (diferencias con BERT)

- **Arquitectura**: GPT apila bloques de **Transformer Decoder**; BERT utiliza **Transformer Encoder**.
- **Objetivo de preentrenamiento**: GPT predice el **siguiente token** (*causal LM*); BERT enmascara tokens (**MLM**) para construir representaciones bidireccionales.
- **Uso típico**: GPT se emplea para **generación de texto**; BERT para **comprensión/representación**.



## 1. Preparación del entorno
Ejecuta la siguiente celda para detectar si estás en Colab y, de ser así, instalar dependencias opcionales.


In [3]:

import pkg_resources, warnings, sys, subprocess, os
warnings.filterwarnings("ignore")

installed = [p.key for p in pkg_resources.working_set]
IN_COLAB = 'google-colab' in installed

def pip_install(requirements):
    print("Instalando dependencias...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "-r", requirements])

if IN_COLAB:
    try:
        import urllib.request
        url = "https://raw.githubusercontent.com/Ohtar10/icesi-nlp/main/requirements.txt"
        local = "requirements.txt"
        urllib.request.urlretrieve(url, local)
        pip_install(local)
    except Exception as e:
        print("No se pudieron instalar dependencias desde el repositorio:", e)

print("IN_COLAB:", IN_COLAB)


IN_COLAB: False



## 2. Cargar el modelo y el tokenizador
Usaremos `DeepESP/gpt2-spanish` y movemos el modelo a GPU si está disponible.


In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

if torch.cuda.is_available():
    device = torch.device("cuda")
elif getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

model_name = "DeepESP/gpt2-spanish"
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

print(f"Usando dispositivo: {device}")
model


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)


### Inspección rápida de módulos


In [5]:

modules = [m for m, _ in model.named_modules()]
modules[:25]  # muestra los primeros 25 para no saturar la salida


['',
 'transformer',
 'transformer.wte',
 'transformer.wpe',
 'transformer.drop',
 'transformer.h',
 'transformer.h.0',
 'transformer.h.0.ln_1',
 'transformer.h.0.attn',
 'transformer.h.0.attn.c_attn',
 'transformer.h.0.attn.c_proj',
 'transformer.h.0.attn.attn_dropout',
 'transformer.h.0.attn.resid_dropout',
 'transformer.h.0.ln_2',
 'transformer.h.0.mlp',
 'transformer.h.0.mlp.c_fc',
 'transformer.h.0.mlp.c_proj',
 'transformer.h.0.mlp.act',
 'transformer.h.0.mlp.dropout',
 'transformer.h.1',
 'transformer.h.1.ln_1',
 'transformer.h.1.attn',
 'transformer.h.1.attn.c_attn',
 'transformer.h.1.attn.c_proj',
 'transformer.h.1.attn.attn_dropout']


## 3. Demostración: *forward pass* y logits
Observamos dimensiones de entrada/salida, extraemos el último vector de logits y calculamos probabilidades de los **top‑k** siguientes tokens.


In [6]:

import torch

prompt = "Había una vez"
best = 10

with torch.no_grad():
    tokens = tokenizer(prompt, return_tensors='pt')['input_ids'].to(device)
    print("Dimensiones de la entrada:", tokens.shape)
    output = model(input_ids=tokens)
    print("Dimensiones de la salida:", output.logits.shape)
    last_logits = output.logits[0, -1, :]
    print("Dimensiones del último token de la secuencia:", last_logits.shape)
    probs = torch.softmax(last_logits, dim=-1)
    print("Dimensiones de las probabilidades:", probs.shape)
    topk = torch.topk(probs, best)
    print({tokenizer.decode(i): f"{p.item()*100:.2f}%" for i, p in zip(topk.indices, topk.values)})


Dimensiones de la entrada: torch.Size([1, 3])
Dimensiones de la salida: torch.Size([1, 3, 50257])
Dimensiones del último token de la secuencia: torch.Size([50257])
Dimensiones de las probabilidades: torch.Size([50257])
{' más': '40.87%', ' que': '10.55%', ' en': '7.27%', ',': '4.90%', ' allí': '0.99%', ' se': '0.87%', '.': '0.82%', ' dentro': '0.81%', ' a': '0.78%', ' al': '0.78%'}



## 4. Función de generación personalizada (*ε-greedy* + muestreo)

- `eps=1.0` → **greedy** puro (siempre el token más probable).
- `eps=0.0` → **muestreo** puro (siempre samplea de la distribución).
- `0 < eps < 1` → mezcla: con prob. `eps` usa greedy; en otro caso, muestrea.


In [7]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from typing import Optional, Tuple
from transformers.tokenization_utils_base import PreTrainedTokenizerBase

def generate(
        model: nn.Module,
        tokenizer: PreTrainedTokenizerBase,
        start: str,
        max_length: int = 100,
        eps: float = 0.5,
        top_n: int = 5,
        return_iterations: bool = False,
        device: Optional[torch.device] = None
    ) -> Tuple[str, Optional[pd.DataFrame]]:

    output = [start]
    iterations = []
    target_device = device if device is not None else next(model.parameters()).device

    with torch.no_grad():
        inputs = tokenizer(output[-1], return_tensors='pt').to(target_device)
        input_ids = inputs['input_ids']
        for _ in range(max_length):
            logits = model(input_ids=input_ids).logits
            probs = torch.softmax(logits[0, -1, :], dim=-1)
            sorted_tokens = torch.argsort(probs, dim=-1, descending=True)

            if np.random.random_sample(1)[0] < eps:
                next_token = sorted_tokens[0].unsqueeze(dim=0)
            else:
                next_token = torch.multinomial(probs, 1)

            if return_iterations:
                step = {'input': ''.join(output)}
                best_n = sorted_tokens[:top_n].cpu().tolist()
                choices = {f'Choice #{i+1}': f"{tokenizer.decode([tok], skip_special_tokens=True)} ({probs[best_n[i]].item():.4f})"
                           for i, tok in enumerate(best_n)}
                step.update(choices)
                iterations.append(step)

            output.append(tokenizer.decode(next_token, skip_special_tokens=True))
            next_token = next_token.unsqueeze(dim=0)
            input_ids = torch.cat([input_ids, next_token], dim=-1)

        text_out = ''.join(output)
        if not return_iterations:
            return text_out, None
        df = pd.DataFrame(iterations)
        return text_out, df



### 4.1 Greedy (ε = 1.0)


In [8]:
legend_prompt = "Cuenta la leyenda que en el valle escondido de Brumalia, "
output_text, iterations_df = generate(
    model,
    tokenizer,
    legend_prompt,
    max_length=25,
    eps=1.0,
    top_n=10,
    return_iterations=True,
    device=device,
)
print(output_text)
iterations_df.head(15)


Había una vez más, el hombre que había sido su padre, el que había sido su


,input,Choice #1,Choice #2,Choice #3,Choice #4,Choice #5,Choice #6,Choice #7,Choice #8,Choice #9,Choice #10
0,Había una vez,más (0.4087),que (0.1055),en (0.0727),", (0.0490)",allí (0.0099),se (0.0087),. (0.0082),dentro (0.0081),a (0.0078),al (0.0078)
1,Había una vez más,", (0.5323)",en (0.0701),. (0.0519),la (0.0194),que (0.0191),: (0.0174),el (0.0162),se (0.0157),a (0.0142),y (0.0113)
2,"Había una vez más,",el (0.0491),se (0.0456),la (0.0409),en (0.0339),y (0.0301),no (0.0299),había (0.0205),me (0.0194),su (0.0158),los (0.0157)
3,"Había una vez más, el",hombre (0.0230),hecho (0.0225),señor (0.0173),joven (0.0165),recuerdo (0.0157),muchacho (0.0120),mundo (0.0119),corazón (0.0105),cuerpo (0.0095),deseo (0.0087)
4,"Había una vez más, el hombre",que (0.1364),se (0.0979),de (0.0590),había (0.0475),no (0.0358),le (0.0276),estaba (0.0216),la (0.0205),del (0.0204),al (0.0195)
5,"Había una vez más, el hombre que",había (0.1757),se (0.0684),estaba (0.0556),le (0.0475),la (0.0471),tenía (0.0348),lo (0.0310),me (0.0221),no (0.0146),amaba (0.0139)
6,"Había una vez más, el hombre que había",sido (0.0862),estado (0.0856),visto (0.0478),hablado (0.0323),conocido (0.0255),intentado (0.0232),hecho (0.0230),matado (0.0214),en (0.0184),entrado (0.0129)
7,"Había una vez más, el hombre que había sido",su (0.1692),", (0.0989)",el (0.0616),. (0.0441),antes (0.0383),en (0.0360),y (0.0337),capaz (0.0231),durante (0.0221),siempre (0.0183)
8,"Había una vez más, el hombre que había sido su",padre (0.1336),compañero (0.1296),amigo (0.1290),hermano (0.0368),marido (0.0357),primer (0.0353),hijo (0.0245),jefe (0.0181),mejor (0.0178),amante (0.0173)
9,"Había una vez más, el hombre que había sido su...",", (0.1805)",había (0.1334),. (0.1022),y (0.0761),se (0.0562),no (0.0505),en (0.0381),le (0.0301),durante (0.0280),era (0.0277)



### 4.2 Mezcla (ε = 0.5)


In [9]:
output_text, _ = generate(
    model,
    tokenizer,
    legend_prompt,
    max_length=120,
    eps=0.5,
    device=device,
)
print(output_text)


Había una vez fuera, probablemente Ollie y ella se convirtieron en estudiantiles de universidad. 

—No sé dónde vivir —dijo Ollie—. Hay dos clases. Ollie es una estudiante de segundo nivel, y no necesita que la llamen para que opere. 

—Lo discutiré contigo —dijo Ollie—. Y te haré una visita. 

—No te preocupes por eso —dijo Ollie—. No es buena idea. 

—Yo también, Florian. 

—No te



## 5. Generación con `model.generate` (Transformers)
Más parámetros: `temperature`, `top_k`, `top_p`, `repetition_penalty`, etc.
Documentación: 
- https://huggingface.co/docs/transformers/main_classes/text_generation#transformers.GenerationConfig
- https://huggingface.co/docs/transformers/main_classes/text_generation#transformers.GenerationMixin.generate


In [10]:
inputs = tokenizer(legend_prompt, return_tensors='pt').to(device)
output = model.generate(
    **inputs,
    pad_token_id=tokenizer.eos_token_id,
    max_length=100,
    do_sample=True,
    temperature=0.7,
    top_k=50,
)
print(tokenizer.decode(output[0], skip_special_tokens=True))


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Había una vez más sin saber qué hacer para detener el impulso de la furia, y en un momento dado se dio cuenta de que si no lo hacía ya lo sabría. 

—Le enseñaré a la policía —dijo—. Yo estoy en la sala de interrogatorios y no he venido aquí para que me encierren. 

—Le enseñaré a la policía —dijo el inspector, señalando la puerta—. Y ahora, si no me van a hacer algo, le diré que me han encerrado aquí. 




## 6. Ajuste fino (fine‑tuning) con un corpus propio

Para ilustrar el flujo, ofrecemos **dos opciones**:
1. **Corpus local**: un archivo `JSONL` con una clave `text` por fila.




In [ ]:

from datasets import load_dataset


data_path = "/Users/j.ricardomunoz/Desktop/Practica 5/leyendas_ficticias.jsonl"
dataset = load_dataset('json', data_files={'train': data_path})


dataset


Generating train split: 0 examples [00:00, ? examples/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Generating train split: 10 examples [00:00, 192.57 examples/s]


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 10
    })
})


### 6.1 EDA rápido


In [13]:

dataset['train'][0]


{'text': 'En el pueblo de Brumalia, cada solsticio se abría una puerta luminosa en la plaza. Los ancianos decían que conducía al taller de los vientos, donde artesanos invisibles tejían corrientes para el nuevo año.'}

In [14]:

dataset.set_format('pandas')
df = dataset['train'].to_pandas()
df['Palabras por registro'] = df['text'].astype(str).str.split().apply(len)
df[['text', 'Palabras por registro']].head()


,text,Palabras por registro
0,"En el pueblo de Brumalia, cada solsticio se ab...",34
1,"Cuando la montaña Mirabal despertó, no fue con...",32
2,La biblioteca sumergida de Salmorán se dejaba ...,31
3,Un puñado de niños descubrió que las luciérnag...,30
4,Cuenta la leyenda que el reloj del faro de Ven...,29


In [15]:

df['Palabras por registro'].describe()


count    10.00000
mean     31.00000
std       2.44949
min      27.00000
25%      29.25000
50%      30.50000
75%      33.50000
max      34.00000
Name: Palabras por registro, dtype: float64


### 6.2 Tokenización y *train/test split*


In [16]:

dataset.reset_format()

def preprocess_function(max_len):
    def _preprocess(examples):
        return tokenizer(examples['text'], max_length=max_len, truncation=True, padding='max_length')
    return _preprocess

tokenized = dataset['train'].map(preprocess_function(max_len=128), batched=True)
tokenized = tokenized.remove_columns([c for c in tokenized.column_names if c != 'input_ids'])
tokenized = tokenized.train_test_split(train_size=0.8, seed=42)
tokenized.set_format('torch')
tokenized


Map: 100%|██████████| 10/10 [00:00<00:00, 383.14 examples/s]


DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 8
    })
    test: Dataset({
        features: ['input_ids'],
        num_rows: 2
    })
})


### 6.3 Entrenamiento
Reducimos `batch_size` y épocas por ser un ejemplo.


In [19]:
from transformers import DataCollatorForLanguageModeling, Trainer, TrainingArguments

batch_size = 2
logging_steps = max(1, len(tokenized['train']) // batch_size)

training_args = TrainingArguments(
    output_dir='./hf-gpt-finetuned',
    overwrite_output_dir=True,
    num_train_epochs=5,
    learning_rate=5e-5,
    per_device_eval_batch_size=batch_size,
    per_device_train_batch_size=batch_size,
    weight_decay=0.01,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    disable_tqdm=False,
    logging_steps=logging_steps,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
    train_dataset=tokenized['train'],
    eval_dataset=tokenized['test'],
    tokenizer=tokenizer,
)

trainer.train()


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 50256}.
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,4.616600,4.763004
2,3.817100,4.653461
3,3.073900,4.594625
4,2.592300,4.571231
5,2.387900,4.569223


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=20, training_loss=3.2975602626800535, metrics={'train_runtime': 68.5571, 'train_samples_per_second': 0.583, 'train_steps_per_second': 0.292, 'total_flos': 2612920320000.0, 'train_loss': 3.2975602626800535, 'epoch': 5.0})


### 6.4 Generación después del ajuste fino


In [21]:
# Si has ejecutado el entrenamiento arriba, el modelo ya está actualizado.
inputs = tokenizer(legend_prompt, return_tensors='pt').to(device)
with torch.no_grad():
    output = model.generate(
        **inputs,
        pad_token_id=tokenizer.eos_token_id,
        max_length=120,
        do_sample=True,
        temperature=0.8,
        top_k=50,
    )
print(tokenizer.decode(output[0], skip_special_tokens=True))


Había una vez que un hombre se acercaba a ella, su corazón latía a mil por hora. 

—Tengo asuntos más urgentes que atender, si quiere tomar represalias. 

—Daré órdenes a los hombres para que se unan a nosotros. 

—No voy a permitir que los demás hagan ningún movimiento hacia mí. 

—El que no quiera que me dejen marchar al campo es un amigo de la familia. Puedo encontrar a un hombre que quiera que me lleve conmigo, pero no puedo hacer nada para impedirlo. 

—Por favor, diles que no se retire


In [22]:
legend_prompts = [
    "Cuenta la leyenda que en el valle escondido de Brumalia, ",
    "Dicen los ancianos de Mirabal que ",
    "En las noches de luna roja de Salmorán, ",
]

samples = []
for prompt in legend_prompts:
    generated, _ = generate(model, tokenizer, prompt, max_length=120, eps=0.3, device=device)
    samples.append({"prompt": prompt, "leyenda_generada": generated})

pd.DataFrame(samples)


Había una vez construidas, ovieres y telescopios trabajaban en silencio. No vivían en el espacio abierto. Cada uno de los dos Estados constituía una parte del sistema. 

—Muy bien —dijo el capitán—. ¿Qué hay del sistema? 

—El del Laboratorio Médico Clínica ocurrió hace unos años. Más tarde, en el año 2002, examinaron los datos de la vida de astronautas en elquina. La información descubrió que el patrón de vida de astronautas era el mismo que creemos. 

—¿Y, entonces? 

—No lo sabemos. 

Gardner no dijo nada,
